In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


from imblearn.over_sampling import RandomOverSampler

from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score,confusion_matrix

In [ ]:
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')

In [ ]:
train_data.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [ ]:
test_data.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,136429,L50896,L,302.3,311.5,1499,38.0,60,0,0,0,0,0
1,136430,L53866,L,301.7,311.0,1713,28.8,17,0,0,0,0,0
2,136431,L50498,L,301.3,310.4,1525,37.7,96,0,0,0,0,0
3,136432,M21232,M,300.1,309.6,1479,47.6,5,0,0,0,0,0
4,136433,M19751,M,303.4,312.3,1515,41.3,114,0,0,0,0,0


In [ ]:
train_data.isnull().sum()

id                         0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

In [ ]:
test_data.isnull().sum()

id                         0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

In [ ]:
train_data['Machine failure'].value_counts()

Machine failure
0    134281
1      2148
Name: count, dtype: int64

In [ ]:
X = train_data .drop(columns=['Machine failure'])
y = train_data ['Machine failure']

# Random Oversampling, cân bằng dữ liệu
oversampler = RandomOverSampler()
X_resampled, y_resampled = oversampler.fit_resample(X, y)

In [ ]:
#đếm số lượng của y_resampled sau khi thực hiện RandomOverSampler
from collections import Counter
Counter(y_resampled)

Counter({0: 134281, 1: 134281})

In [ ]:
X_resampled['Temperature Difference'] = X_resampled['Air temperature [K]'] - X_resampled['Process temperature [K]']

In [ ]:
X_resampled.drop(columns=['id', 'Product ID'], inplace=True)

In [ ]:
label_encoder = LabelEncoder()

X_resampled['Type'] = label_encoder.fit_transform(X_resampled['Type'])

In [ ]:
numerical_features = ["Air temperature [K]", "Process temperature [K]", "Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]

In [ ]:
scaler = StandardScaler()
X_resampled[numerical_features] = scaler.fit_transform(X_resampled[numerical_features])

In [ ]:
X_resampled

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF,Temperature Difference
0,1,0.120510,-0.383785,0.484673,-0.749792,0.306877,0,0,0,0,0,-9.0
1,2,1.121949,1.493048,1.210722,-1.327383,1.159781,0,0,0,0,0,-9.5
2,1,-0.530425,-1.209591,1.415619,-1.541917,-1.327855,0,0,0,0,0,-9.2
3,1,0.320798,0.592168,0.163964,-0.073185,1.117136,0,0,0,0,0,-9.9
4,2,-1.181361,-0.834225,0.685116,-0.807551,-1.199920,0,0,0,0,0,-11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
268557,0,1.071877,0.592168,-0.584356,0.850961,-0.247510,0,0,0,0,0,-8.4
268558,2,0.020366,1.192755,-0.700167,1.164510,0.974985,0,0,0,1,0,-11.3
268559,1,-1.682080,-1.735104,-0.766982,1.321285,1.159781,1,0,0,1,0,-10.8
268560,1,0.721373,0.066655,-0.570993,0.141349,-1.569511,0,1,0,0,0,-8.4


In [ ]:
y_resampled

0         0
1         0
2         0
3         0
4         0
         ..
268557    1
268558    1
268559    1
268560    1
268561    1
Name: Machine failure, Length: 268562, dtype: int64

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
models = {
    'RandomForest': RandomForestClassifier(),
    'SVC': SVC(),
    'LogisticRegression': LogisticRegression(),
    'KNeighbors': KNeighborsClassifier()
}

In [ ]:
cv_scores= {}

In [ ]:
for name, model in models.items():
    print("name model : ",name)
    scores = cross_val_score(model, X_resampled[:10000], y_resampled[:10000], cv=5, scoring='accuracy')
    cv_scores[name] = scores.mean()
    print(cv_scores[name])

name model :  RandomForest
0.9966000000000002
name model :  SVC
0.9856999999999999
name model :  LogisticRegression


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

0.9953
name model :  KNeighbors
0.9907


In [ ]:
for name, score in cv_scores.items():
    print(f'{name}: {score}')

RandomForest: 0.9966000000000002
SVC: 0.9856999999999999
LogisticRegression: 0.9953
KNeighbors: 0.9907


In [ ]:
best_model_name = max(cv_scores, key=cv_scores.get)
print(f'Best model: {best_model_name} with mean cross-validation score of {cv_scores[best_model_name]}')

Best model: RandomForest with mean cross-validation score of 0.9966000000000002


In [ ]:
best_model_name

'RandomForest'

In [ ]:
best_model = models[best_model_name]
best_model

RandomForestClassifier()

In [ ]:
best_model.fit(X_resampled, y_resampled)
train_predictions = best_model.predict(X_resampled)

In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90954 entries, 0 to 90953
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       90954 non-null  int64  
 1   Product ID               90954 non-null  object 
 2   Type                     90954 non-null  object 
 3   Air temperature [K]      90954 non-null  float64
 4   Process temperature [K]  90954 non-null  float64
 5   Rotational speed [rpm]   90954 non-null  int64  
 6   Torque [Nm]              90954 non-null  float64
 7   Tool wear [min]          90954 non-null  int64  
 8   TWF                      90954 non-null  int64  
 9   HDF                      90954 non-null  int64  
 10  PWF                      90954 non-null  int64  
 11  OSF                      90954 non-null  int64  
 12  RNF                      90954 non-null  int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 9.0+ MB


In [ ]:
test_data_id = test_data['id']

# Feature Engineering (Temperature Difference)
test_data['Temperature Difference'] = test_data['Air temperature [K]'] - test_data['Process temperature [K]']

# Drop 'id' and 'Product ID' columns
test_data.drop(columns=['id', 'Product ID'], inplace=True)

test_data['Type'] = label_encoder.transform(test_data['Type'])

# Scaling numerical features
test_data[numerical_features] = scaler.transform(test_data[numerical_features])

# Make predictions on the test data
test_predictions = best_model.predict(test_data)

In [ ]:
predictions_df = pd.DataFrame({'id':test_data_id ,
                               'Machine failure': test_predictions})

In [ ]:
predictions_df.to_csv("predictions.csv", index=False)